In [6]:
import pandas as pd

# Carrega o CSV
df = pd.read_csv('data/txu_samples_processado.csv')

# Remove a coluna pelo nome
df = df.drop(columns=['id', 'linha_original'])
df.to_csv('data/txu_samples.csv', index=False)

# Salva o CSV sem a coluna
df.columns

Index(['kayapo', 'ingles'], dtype='object')

In [1]:
import pandas as pd

df = pd.read_csv("evaluation_results.csv")
for idx, row in df.sample(10, random_state=42).iterrows():
    print(f"REF: {row.reference}")
    print(f"PRED: {row.prediction}\n")

SyntaxError: incomplete input (1465740890.py, line 6)

In [11]:
convert_to_txt("data/txuNT_all.pdf", "data/txu.txt")

In [69]:
def parallelize(txt1, txt2, res_file):
    with open(txt1, encoding="utf-8") as f_en, open(txt2, encoding="utf-8") as f_txu:
        eng_lines = f_en.readlines()
        txu_lines = f_txu.readlines()

    with open(res_file, "w", encoding="utf-8") as out:
        for en, tx in zip(eng_lines, txu_lines):
            out.write(f"{en.strip()}\t{tx.strip()}\n")
        
parallelize("data/eng.txt", "data/txu.txt", "data/parallel.txt")

In [1]:
eng_data = []
txu_data = []
with open("data/parallel.txt", "r") as f:
    for line in f:
        line.strip('\n')
        eng, txu = line.split('\t')
        eng_data.append(eng)
        txu_data.append(txu)
        
eng_data[5]
txu_data[5]

'copyright © 2012 Wycliffe Bible Translators, Inc.\n'

In [2]:
from datasets import load_dataset

data = load_dataset(
    "csv",
    data_files="data/parallel.txt",
    delimiter="\t",
    column_names=["en","txu"],
    split="train"
)

data = data.train_test_split(test_size=0.1)

In [3]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['en', 'txu'],
        num_rows: 54225
    })
    test: Dataset({
        features: ['en', 'txu'],
        num_rows: 6026
    })
})


In [4]:
from datasets import DatasetDict
import nlpaug.augmenter.word as naw

train_dataset = data['train']
test_dataset = data['test']
aug = naw.SynonymAug(aug_src='wordnet')

def augment_batch(batch):
    return {
        'en': [aug.augment(text) for text in batch['en']],
        'txu': batch['txu']
    }

train_aug_dataset = train_dataset.map(augment_batch, batched=True, batch_size=32)
test_aug_dataset = test_dataset.map(augment_batch, batched=True, batch_size=32)

Map:   0%|          | 0/54225 [00:00<?, ? examples/s]

Map:   0%|          | 0/6026 [00:00<?, ? examples/s]

In [5]:
from datasets import concatenate_datasets

def join_en(example):
    if isinstance(example["en"], list):
        return {"en": " ".join(example["en"])}
    return example

train_aug_dataset = train_aug_dataset.map(join_en)
test_aug_dataset = test_aug_dataset.map(join_en)

train_dataset = concatenate_datasets([train_dataset, train_aug_dataset])
test_dataset = concatenate_datasets([test_dataset, test_aug_dataset])

Map:   0%|          | 0/54225 [00:00<?, ? examples/s]

Map:   0%|          | 0/6026 [00:00<?, ? examples/s]

In [6]:
import nlpaug.augmenter.word as naw

aug = naw.RandomWordAug(action="swap")

def augment_batch(batch):
    return {
        'en': [aug.augment(text) for text in batch['en']],
        'txu': batch['txu']
    }

train_aug_dataset = train_dataset.map(augment_batch, batched=True, batch_size=32)
test_aug_dataset = test_dataset.map(augment_batch, batched=True, batch_size=32)

Map:   0%|          | 0/108450 [00:00<?, ? examples/s]

Map:   0%|          | 0/12052 [00:00<?, ? examples/s]

In [7]:


train_dataset = concatenate_datasets([train_dataset, train_aug_dataset])
test_dataset = concatenate_datasets([test_dataset, test_aug_dataset])

Map:   0%|          | 0/108450 [00:00<?, ? examples/s]

Map:   0%|          | 0/12052 [00:00<?, ? examples/s]

In [8]:
train_dataset

Dataset({
    features: ['en', 'txu'],
    num_rows: 216900
})

In [9]:
from datasets import DatasetDict

dataset = DatasetDict({
    'train': train_dataset.shuffle(seed=42),
    'test': data['test']
})

dataset.save_to_disk("augmented_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/216900 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6026 [00:00<?, ? examples/s]

In [1]:
import os
import warnings

# Resolver problema do tokenizers|
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Desabilitar alguns warnings
warnings.filterwarnings("ignore")

In [2]:
from transformers.integrations import deepspeed
deepspeed._hf_deepspeed_config_weak_ref = None

from datasets import load_from_disk
dataset = load_from_disk("augmented_dataset")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['en', 'txu'],
        num_rows: 216900
    })
    test: Dataset({
        features: ['en', 'txu'],
        num_rows: 6026
    })
})


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from datasets import load_dataset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_FP16 = torch.cuda.is_available()
model_name = "Helsinki-NLP/opus-mt-en-ROMANCE"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)

# Carrega dataset e divide em treino/teste
raw_dataset = load_dataset("csv", data_files="data/parallel.txt", delimiter="\t", column_names=["en", "txu"], split="train")
dataset = raw_dataset.train_test_split(test_size=0.1)

# Filtra entradas nulas em ambos splits
dataset["train"] = dataset["train"].filter(lambda x: x["en"] is not None and x["txu"] is not None)
dataset["test"] = dataset["test"].filter(lambda x: x["en"] is not None and x["txu"] is not None)

# Pré-processamento
def preprocess(example):
    inputs = tokenizer(example["en"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(example["txu"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = dataset["train"].map(preprocess, batched=True)
test_dataset = dataset["test"].map(preprocess, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

Filter:   0%|          | 0/54225 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6026 [00:00<?, ? examples/s]

Map:   0%|          | 0/41544 [00:00<?, ? examples/s]

Map:   0%|          | 0/4584 [00:00<?, ? examples/s]

In [6]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['en', 'txu', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 41544
})
Dataset({
    features: ['en', 'txu', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4584
})


In [18]:
from accelerate import Accelerator
import gc
import os
import torch

def clear_memory():
    """Clear GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

def print_memory_usage():
    """Print current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**2
        reserved = torch.cuda.memory_reserved() / 1024**2
        print(f"GPU Memory - Allocated: {allocated:.2f} MB, Reserved: {reserved:.2f} MB")

clear_memory()

# Initialize accelerator with memory optimizations
accelerator = Accelerator(
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
    mixed_precision="fp16"  # Use half precision to save memory
)

In [19]:
import evaluate

rouge = evaluate.load("rouge")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: v.mid.fmeasure * 100 for k, v in result.items()}

In [ ]:
import os
from transformers import (
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import torch
from peft import LoraConfig, get_peft_model

model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M",  # Use half precision
    low_cpu_mem_usage=True,
    device_map="auto"
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.config.use_cache = False

model.gradient_checkpointing_enable()

# Add LoRA adapters instead of full fine-tuning
lora_config = LoraConfig(
    r=8,  # Low rank
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt",
    max_length=64  # Limit sequence length to save memory
)


In [17]:
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

CUDA disponível: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Memória GPU: 4.00 GB


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=1,  # Keep batch size at 1
    shuffle=True,
    collate_fn=data_collator,
    pin_memory=False,  # Disable pin_memory to save GPU memory
    num_workers=0  # Disable multiprocessing to save memory
)

eval_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=1,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=False,
    num_workers=0
)

optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=3e-5,  # Slightly lower learning rate
    weight_decay=0.01
)


In [10]:

num_epochs = 3
num_training_steps = num_epochs * len(train_loader) // accelerator.gradient_accumulation_steps

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_training_steps // 10,
    num_training_steps=num_training_steps,
)

model, optimizer, train_loader, eval_loader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_loader, eval_loader, lr_scheduler
)

In [11]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print_memory_usage()

print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for step, batch in enumerate(train_loader):
        with accelerator.accumulate(model):
            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss / accelerator.gradient_accumulation_steps
            
            # Backward pass
            accelerator.backward(loss)
            
            # Optimizer step (only when gradients are accumulated)
            if accelerator.sync_gradients:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                
                # Clear memory periodically
                if step % 50 == 0:
                    clear_memory()
            
            total_loss += loss.item()
        
        # Print progress
        if step % 100 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")
            print_memory_usage()
    
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train loss: {avg_train_loss:.4f}")
    
    # Validation with memory management
    model.eval()
    eval_loss = 0
    
    with torch.no_grad():
        for step, batch in enumerate(eval_loader):
            outputs = model(**batch)
            eval_loss += outputs.loss.item()
            
            # Clear memory during validation too
            if step % 50 == 0:
                clear_memory()
    
    avg_eval_loss = eval_loss / len(eval_loader)
    print(f"Epoch {epoch+1} | Eval loss: {avg_eval_loss:.4f}")
    print_memory_usage()
    
    # Clear memory after each epoch
    clear_memory()

print("Training completed!")


GPU Memory - Allocated: 2358.84 MB, Reserved: 5368.00 MB
Starting training...


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
def load_smaller_model():
    """Load a smaller NLLB model if the 600M version is too large"""
    clear_memory()
    
    # Try the 200M version instead
    model_name = "facebook/nllb-200-200M"  # Much smaller model
    
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto"
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return model, tokenizer

# Uncomment the following lines if you still get OOM errors:
# print("Loading smaller model due to memory constraints...")
# model, tokenizer = load_smaller_model()

In [ ]:
trainer.save_model("./modelo_final")
tokenizer.save_pretrained("./modelo_final")